# Replicating Moss et al. 2019

[@moss2019WhatCanUrban]


In [ ]:
using DrWatson
using CSV

get the mixing matrix from CSV

In [ ]:
frac_self = 0.5
frac_cbd = 0.2
path = joinpath(datadir(), "Moss2018\\mixing\\abs_all.csv")
mixmat =CSV.File(path) 
CSV.ta

Epidemiological parameters


In [ ]:
r0 = 1.4
σ = 2.0
γ = 0.5

Model Parameter type


In [ ]:
struct Params
    β::Float64
    σ::Float64
    γ::Float64
    popn::Vector
    mixmat::AbstractArray
end

Model State type

In [ ]:
struct Model
    params::Params
    numpatch::Int
    t::Float64
    s::Vector
    e::Vector
    i::Vector
end

Event type
